In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE117589"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE117589"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE117589.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE117589.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE117589.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"REST and Neural Gene Network Dysregulation in iPS Cell Models of Alzheimer’s Disease"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: induced pluripotent stem cells', 'cell type: neurons', 'cell type: neural progenitor cells'], 1: ['subject: 60F', 'subject: 64M', 'subject: 72M', 'subject: 73M', 'subject: 75F', 'subject: 92F', 'subject: 60M', 'subject: 69F', 'subject: 87F'], 2: ['diagnosis: normal', "diagnosis: sporadic Alzheimer's disease"], 3: ['clone: Clone 1', 'clone: Clone 2'], 4: ['coriell #: AG04455', 'coriell #: AG08125', 'coriell #: AG08379', 'coriell #: AG08509', 'coriell #: AG14244', 'coriell #: AG09173', 'coriell #: AG07376', 'coriell #: AG21158', 'coriell #: AG08243', 'coriell #: AG10788', 'coriell #: AG06869']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information and sample characteristics, this appears to be a dataset with gene expression data
# from iPSCs, neurons, and neural progenitor cells. Therefore, gene expression data is likely available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For Alzheimer's Disease trait:
# Looking at key 2, we see "diagnosis: normal" and "diagnosis: sporadic Alzheimer's disease"
trait_row = 2

# For age:
# Age is not explicitly given but might be inferred from key 1 where subject info contains age and gender
# e.g., 'subject: 60F', 'subject: 64M'
age_row = 1

# For gender:
# Gender is also in key 1 as part of subject information
gender_row = 1

# 2.2 Data Type Conversion

def convert_trait(value):
    if not isinstance(value, str):
        return None
    value = value.split(': ')[-1].strip().lower()
    if "alzheimer" in value or "ad" in value:
        return 1
    elif "normal" in value or "control" in value or "healthy" in value:
        return 0
    return None

def convert_age(value):
    if not isinstance(value, str):
        return None
    # Extract age from patterns like 'subject: 60F', 'subject: 64M'
    value = value.split(': ')[-1].strip()
    # Extract digits from the beginning of the string
    import re
    age_match = re.match(r'^(\d+)', value)
    if age_match:
        try:
            return int(age_match.group(1))
        except ValueError:
            return None
    return None

def convert_gender(value):
    if not isinstance(value, str):
        return None
    # Extract gender from patterns like 'subject: 60F', 'subject: 64M'
    value = value.split(': ')[-1].strip()
    # Check if the last character is 'F' or 'M'
    if value.endswith('F'):
        return 0  # Female
    elif value.endswith('M'):
        return 1  # Male
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assume clinical_data is already loaded from a previous step
    try:
        # Extract clinical features using the clinical_data DataFrame from step 1
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        print("Preview of extracted clinical features:")
        print(preview_df(clinical_features))
        
        # Save the extracted clinical features
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except NameError:
        print("Clinical data not available from previous steps. Skipping clinical feature extraction.")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")


Preview of extracted clinical features:
{'GSM3304268': [0.0, 60.0, 0.0], 'GSM3304269': [0.0, 64.0, 1.0], 'GSM3304270': [0.0, 72.0, 1.0], 'GSM3304271': [0.0, 73.0, 1.0], 'GSM3304272': [0.0, 75.0, 0.0], 'GSM3304273': [0.0, 92.0, 0.0], 'GSM3304274': [1.0, 60.0, 1.0], 'GSM3304275': [1.0, 69.0, 0.0], 'GSM3304276': [1.0, 72.0, 1.0], 'GSM3304277': [1.0, 87.0, 0.0], 'GSM3304278': [0.0, 60.0, 0.0], 'GSM3304279': [0.0, 64.0, 1.0], 'GSM3304280': [0.0, 72.0, 1.0], 'GSM3304281': [0.0, 73.0, 1.0], 'GSM3304282': [0.0, 75.0, 0.0], 'GSM3304283': [0.0, 92.0, 0.0], 'GSM3304284': [1.0, 60.0, 0.0], 'GSM3304285': [1.0, 60.0, 1.0], 'GSM3304286': [1.0, 69.0, 0.0], 'GSM3304287': [1.0, 72.0, 1.0], 'GSM3304288': [1.0, 87.0, 0.0], 'GSM3304289': [0.0, 60.0, 0.0], 'GSM3304290': [0.0, 64.0, 1.0], 'GSM3304291': [0.0, 72.0, 1.0], 'GSM3304292': [0.0, 73.0, 1.0], 'GSM3304293': [0.0, 92.0, 0.0], 'GSM3304294': [1.0, 60.0, 0.0], 'GSM3304295': [1.0, 60.0, 1.0], 'GSM3304296': [1.0, 69.0, 0.0], 'GSM3304297': [1.0, 72.0, 1.0],

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at',
       'ENSG00000000457_at', 'ENSG00000000460_at', 'ENSG00000000938_at',
       'ENSG00000000971_at', 'ENSG00000001036_at', 'ENSG00000001084_at',
       'ENSG00000001167_at', 'ENSG00000001460_at', 'ENSG00000001461_at',
       'ENSG00000001497_at', 'ENSG00000001561_at', 'ENSG00000001617_at',
       'ENSG00000001626_at', 'ENSG00000001629_at', 'ENSG00000001631_at',
       'ENSG00000002016_at', 'ENSG00000002079_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analysis of gene identifiers
# The identifiers start with 'ENSG' which indicates they are Ensembl gene IDs
# These are not standard human gene symbols (like BRCA1, APP, etc.)
# Ensembl IDs need to be mapped to standard gene symbols for better interpretability

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at', 'ENSG00000000457_at', 'ENSG00000000460_at'], 'SPOT_ID': ['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460'], 'Description': ['tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]', 'tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]', 'dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]', 'SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]', 'chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain gene identifiers and gene symbols
# The 'ID' column in gene_annotation matches the index in gene_data
# We need to extract the official gene symbols from the Description field

# Let's look at the SPOT_ID and Description columns more closely
print("Sample SPOT_ID and Description pairs:")
for i in range(min(5, len(gene_annotation))):
    print(f"SPOT_ID: {gene_annotation.iloc[i]['SPOT_ID']} - Description: {gene_annotation.iloc[i]['Description']}")

# Create a mapping from ENSEMBL IDs to gene symbols using regex to extract symbols from Description
import re

def extract_gene_symbol_from_description(description_text):
    if not isinstance(description_text, str):
        return []
    
    # Pattern to extract HGNC symbols from description
    # Example: "tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]" -> extract the HGNC ID 11858
    hgnc_match = re.search(r'HGNC:(\d+)', description_text)
    if hgnc_match:
        # Use extract_human_gene_symbols to get any gene symbols in the text
        symbols = extract_human_gene_symbols(description_text)
        if symbols:
            return symbols
        
        # If no symbols found with extract_human_gene_symbols, try to get the first word
        # that might be a gene symbol
        first_part_match = re.match(r'^(\w+)', description_text)
        if first_part_match:
            return [first_part_match.group(1)]
    
    return []

# Create a custom mapping dataframe that contains both ENSEMBL IDs and symbol information
mapping_df = pd.DataFrame({
    'ID': gene_annotation['ID'],
    'Gene': gene_annotation['Description'].apply(extract_human_gene_symbols)
})

# Filter out rows where Gene is an empty list
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]

# Print the first few rows of the mapping to verify
print("Gene mapping preview:")
print(preview_df(mapping_df))
print(f"Number of probes with gene symbols: {len(mapping_df)}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print shape before normalization
print(f"Gene data shape before normalization: {gene_data.shape}")

# Check if gene symbols need normalization
print("Sample gene symbols before normalization:")
print(list(gene_data.index[:10]))

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print shape after normalization
print(f"Gene data shape after normalization: {gene_data.shape}")

# Preview the first few rows of the processed gene expression data
print("\nProcessed gene expression data preview (first 5 rows, 5 columns):")
if not gene_data.empty:
    print(gene_data.iloc[:5, :5])
else:
    print("Gene data is empty after processing")

# Save the processed gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Processed gene data saved to {out_gene_data_file}")


Sample SPOT_ID and Description pairs:
SPOT_ID: ENSG00000000003 - Description: tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]
SPOT_ID: ENSG00000000005 - Description: tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]
SPOT_ID: ENSG00000000419 - Description: dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]
SPOT_ID: ENSG00000000457 - Description: SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]
SPOT_ID: ENSG00000000460 - Description: chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]


Gene mapping preview:
{'ID': ['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at', 'ENSG00000000457_at', 'ENSG00000000460_at'], 'Gene': [['HGNC'], ['HGNC'], ['HGNC'], ['SCY1', 'HGNC'], ['HGNC']]}
Number of probes with gene symbols: 18144
Gene data shape before normalization: (0, 31)
Sample gene symbols before normalization:
[]
Gene data shape after normalization: (0, 31)

Processed gene expression data preview (first 5 rows, 5 columns):
Gene data is empty after processing
Processed gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE117589.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Extract gene symbols from Description field properly
def extract_gene_symbol_from_description(description_text):
    if not isinstance(description_text, str):
        return []
    
    # Get the gene name from the beginning of description (before [Source:...])
    # Example: "tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]" -> "tetraspanin 6"
    name_part = description_text.split('[Source:')[0].strip()
    
    # Many descriptions have format "Gene Name [Source:...]" - extract the gene symbol
    # Gene symbols are typically uppercase, so look for capital letters
    symbols = extract_human_gene_symbols(description_text)
    
    # If we found symbols using the extract_human_gene_symbols function, return them
    if symbols:
        return symbols
    
    # Fallback: try to extract the first word if it looks like a gene symbol
    words = name_part.split()
    if words and len(words[0]) <= 10 and any(c.isupper() for c in words[0]):
        return [words[0]]
    
    return []

# Create a custom mapping dataframe
mapping_df = pd.DataFrame({
    'ID': gene_annotation['ID'],
    'Gene': gene_annotation['Description'].apply(extract_gene_symbol_from_description)
})

# Filter out rows where Gene is an empty list
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]

# Print the first few rows of the mapping to verify
print("Gene mapping preview:")
print(preview_df(mapping_df))
print(f"Number of probes with gene symbols: {len(mapping_df)}")

# Let's also check gene expression data to make sure it's not empty
print("\nGene expression data preview:")
print(f"Gene expression data shape: {gene_data.shape}")
print(f"Sample column names: {list(gene_data.columns[:5])}")

# Extract gene expression data again from the matrix file to ensure we have good data
gene_data = get_genetic_data(matrix_file)
print(f"Re-loaded gene data shape: {gene_data.shape}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene data shape after mapping: {gene_data_mapped.shape}")

# If the mapped data is too small or empty, try a different approach
if gene_data_mapped.shape[0] < 100:
    print("Mapped gene data is suspiciously small. Trying alternative approach...")
    # Direct approach: Extract gene name from the beginning of the Description
    mapping_df = pd.DataFrame({
        'ID': gene_annotation['ID'],
        'Gene': gene_annotation['Description'].apply(lambda x: 
            x.split('[')[0].strip() if isinstance(x, str) else '')
    })
    # Keep only non-empty gene names
    mapping_df = mapping_df[mapping_df['Gene'] != '']
    print(f"Alternative mapping created with {len(mapping_df)} entries")
    
    # Apply alternative mapping
    gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
    print(f"Gene data shape after alternative mapping: {gene_data_mapped.shape}")

# If still empty, use the original gene data with ENSEMBL IDs as gene names
if gene_data_mapped.shape[0] < 100:
    print("Using original gene data with ENSEMBL IDs as fallback")
    # Remove the _at suffix from the index
    gene_data.index = gene_data.index.str.replace('_at', '')
    gene_data_mapped = gene_data
    print(f"Using original gene data: {gene_data_mapped.shape}")

# Save the processed gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_mapped.to_csv(out_gene_data_file)
print(f"Processed gene data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at', 'ENSG00000000457_at', 'ENSG00000000460_at'], 'Gene': [['HGNC'], ['HGNC'], ['HGNC'], ['SCY1', 'HGNC'], ['HGNC']]}
Number of probes with gene symbols: 18145

Gene expression data preview:
Gene expression data shape: (0, 31)
Sample column names: ['GSM3304268', 'GSM3304269', 'GSM3304270', 'GSM3304271', 'GSM3304272']
Re-loaded gene data shape: (20027, 31)
Gene data shape after mapping: (0, 31)
Mapped gene data is suspiciously small. Trying alternative approach...
Alternative mapping created with 18146 entries


Gene data shape after alternative mapping: (2551, 31)
Processed gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE117589.csv


### Step 8: Data Normalization and Linking

In [9]:
# Let's continue from where we left off with the gene data processing
# Load clinical data that was saved earlier
clinical_data = pd.read_csv(out_clinical_data_file)
print("Clinical data columns:", clinical_data.columns)

# Load gene expression data 
gene_data_mapped = pd.read_csv(out_gene_data_file, index_col=0)
print("Gene data shape:", gene_data_mapped.shape)

# We need to transform clinical data into the right format for linking
# First, check if the clinical data has any column that we can use as sample identifiers
if 'Unnamed: 0' in clinical_data.columns:
    clinical_data.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True)
    clinical_data.set_index('Sample', inplace=True)
else:
    # Create a DataFrame with the appropriate structure: samples as columns, features as rows
    # First get sample IDs from gene data
    sample_ids = gene_data_mapped.columns.tolist()
    
    # Create a new DataFrame with the right structure
    new_clinical_df = pd.DataFrame(index=[trait, 'Age', 'Gender'], columns=sample_ids)
    
    # Fill in the values - assuming clinical_data has the same order of samples
    if len(clinical_data) == len(sample_ids):
        for i, sample_id in enumerate(sample_ids):
            if i < len(clinical_data):
                # Get values from clinical_data row i
                row = clinical_data.iloc[i]
                # Assign values to the new DataFrame
                if trait in row:
                    new_clinical_df.loc[trait, sample_id] = row[trait]
                if 'Age' in row:
                    new_clinical_df.loc['Age', sample_id] = row['Age']
                if 'Gender' in row:
                    new_clinical_df.loc['Gender', sample_id] = row['Gender']
    
    clinical_data = new_clinical_df

# 2. Link clinical and genetic data
linked_data = pd.concat([clinical_data, gene_data_mapped], axis=0)
print("Linked data shape:", linked_data.shape)
print("Linked data index preview:", list(linked_data.index[:10]))

# Transpose the linked data to have samples as rows and features as columns
linked_data = linked_data.T
print("Transposed linked data shape:", linked_data.shape)
print("Actual columns in linked_data:", linked_data.columns.tolist()[:10])

# 3. Handle missing values - use the trait variable from environment setup
linked_data = handle_missing_values(linked_data, trait)
print("Data shape after handling missing values:", linked_data.shape)

# 4. Determine trait and demographic bias
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Trait bias assessment: {is_biased}")
print("Data columns after bias assessment:", list(linked_data.columns[:10]))

# 5. Final quality validation and saving metadata
note = "Used alternative gene mapping approach to extract gene symbols from descriptions."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path, 
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset not usable due to bias or other issues. Linked data not saved.")

Clinical data columns: Index(['GSM3304268', 'GSM3304269', 'GSM3304270', 'GSM3304271', 'GSM3304272',
       'GSM3304273', 'GSM3304274', 'GSM3304275', 'GSM3304276', 'GSM3304277',
       'GSM3304278', 'GSM3304279', 'GSM3304280', 'GSM3304281', 'GSM3304282',
       'GSM3304283', 'GSM3304284', 'GSM3304285', 'GSM3304286', 'GSM3304287',
       'GSM3304288', 'GSM3304289', 'GSM3304290', 'GSM3304291', 'GSM3304292',
       'GSM3304293', 'GSM3304294', 'GSM3304295', 'GSM3304296', 'GSM3304297',
       'GSM3304298'],
      dtype='object')
Gene data shape: (2551, 31)
Linked data shape: (2554, 31)
Linked data index preview: ['Alzheimers_Disease', 'Age', 'Gender', 'A-', 'A-52', 'A0', 'A1', 'A10', 'A11', 'A12']
Transposed linked data shape: (31, 2554)
Actual columns in linked_data: ['Alzheimers_Disease', 'Age', 'Gender', 'A-', 'A-52', 'A0', 'A1', 'A10', 'A11', 'A12']
Data shape after handling missing values: (0, 2)
Quartiles for 'Alzheimers_Disease':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max:

/tmp/ipykernel_51556/2649569560.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  linked_data = pd.concat([clinical_data, gene_data_mapped], axis=0)
